### Install OpenAI pakcages

In [ ]:
!pip uninstall --yes openai
!pip install --upgrade pip
!pip install openai 

In [ ]:
!python --version

In [14]:
import os
import openai

openai.api_key = <OPEN_AI_API_KEY>

#### Fine-Tune OpenAI GPT3.5 model
Step 1. generate >50 sample lottie animation instructions and save them as a JSON file named generated_prompt.json <br>
Step 2. generate lottie files with GPT 4 model based on insturction in step 1<br>
Step 3. manually import the generated lottie files into AE, and exportthem again (only keep 10-20 valid samples, other are not regonizable)<br>
Step 4. construct fine-tune dataset with prompt and valid samples<br>
Step 5. fine-tune openAI GPT3.5 with dataset<br>
Step 6. apply fine-tuned model to generate new lottie animation files<br>

In [14]:
def generate_lottie_prompt(action):
    completion = openai.ChatCompletion.create(
    model="gpt-4",
    messages=[
        {"role": "system", "content": "You are lottie file generator. lottie file should be inside code block ```json ```. format of lottie file: version is 5.12.1. layer is required. one shape should be in layer. a, p, o, r, s should be in both layer and shape,  hd = false. ip, op, sr, st should be in top level, layer, and shape. fill and transform should be parallel to shape json. for keyframe,  it should has two json objects with field i,o,s,t,ti,to."},
        {"role": "user", "content": action}
    ]
    )

    return completion.choices[0].message.content


extract json blob from GenAI response

In [10]:
import re
import json

def extract_json_blob(paragraph):
    # Define a regular expression pattern to match the JSON blob
    pattern = r"```json\n(.+?)\n```"

    # Use the re.search() function to find the first match of the pattern in the paragraph
    match = re.search(pattern, paragraph, re.DOTALL)

    if match:
        # If a match is found, extract the JSON blob from the match object
        json_blob = match.group(1)

        # Use the json.loads() function to parse the JSON blob into a Python object
        json_obj = json.loads(json_blob)

        # Return the Python object
        return json_obj
    else:
        # If no match is found, return None
        return None

generate lottie files with gpt4

In [20]:
import json

with open('generated_prompt.json', 'r') as f:
    o = json.load(f)
    for json_obj in o['lottie_prompts'][:72]:
        lo = generate_lottie_prompt(json_obj['description'])
        try:
            lo_extracted = extract_json_blob(lo)
            print(lo_extracted)
            if lo_extracted:
                with open(f"out/{json_obj['id']}.json", 'w') as outfile:
                    json.dump(lo_extracted, outfile)
        except Exception as e:
            print(e)


{'v': '5.12.1', 'ip': 0, 'op': 60, 'fr': 30, 'sr': 1, 'st': 0, 'layers': [{'ddd': 0, 'ind': 1, 'ty': 4, 'nm': 'Rectangle', 'hd': False, 'sr': 1, 'st': 0, 'ip': 0, 'op': 60, 'shapes': [{'ty': 'gr', 'it': [{'ty': 'fl', 'c': {'a': 0, 'k': [0.878, 0.69, 1, 1]}, 'o': {'a': 0, 'k': 100}}, {'ty': 'tr', 'p': {'a': 0, 'k': [0, 0]}, 'sk': {'a': 1, 'k': [{'i': {'x': [0], 'y': [1]}, 'o': {'x': [1], 'y': [0]}, 's': [0], 't': 0, 'to': [0, 0], 'ti': [0, 0]}, {'i': {'x': [0], 'y': [1]}, 'o': {'x': [1], 'y': [0]}, 's': [360], 't': 60, 'to': [0, 0], 'ti': [0, 0]}]}}, {'ty': 'rc', 's': {'a': 1, 'k': [{'i': {'x': [0, 0], 'y': [1, 1]}, 'o': {'x': [1, 1], 'y': [0, 0]}, 's': [0, 0], 't': 0, 'to': [100, 100], 'ti': [0, 0]}, {'i': {'x': [0, 0], 'y': [1, 1]}, 'o': {'x': [1, 1], 'y': [0, 0]}, 's': [100, 100], 't': 1, 'to': [0, 0], 'ti': [100, 100]}]}}]}]}]}
None
{'v': '5.12.1', 'fr': 30, 'ip': 0, 'op': 60, 'sr': 1, 'st': 0, 'hd': False, 'assets': [], 'layers': [{'ddd': 0, 'ind': 1, 'ty': 4, 'nm': 'Ellipse Layer'

create json blob 

In [25]:
import json
import os


def create_json_object(number):
    with open('./out/' + str(number) + '_ae.json', 'r') as f:
        data = json.load(f)
        with open('generated_prompt.json', 'r') as lottie:
            content = json.load(lottie)['lottie_prompts'][int(number)-1]['description']
            json_obj = {"messages": [
                            {"role": "system", "content": "You are lottie file generator. lottie file should be inside code block ```json ```. format of lottie file: version is 5.12.1. layer is required. one shape should be in layer. a, p, o, r, s should be in both layer and shape,  hd = false. ip, op, sr, st should be in top level, layer, and shape. fill and transform should be parallel to shape json. for keyframe,  it should has two json objects with field i,o,s,t,ti,to."}, 
                            {"role": "user", "content": content}, 
                            {"role": "assistant", "content": json.dumps(data, ensure_ascii=False)}
                        ]}
        
        return json_obj


def create_jsonl(json_obj, output_file_path):
    with open(output_file_path, 'a+') as f:
        json.dump(json_obj, f)
        f.write('\n')

In [26]:
for i in [1, 2, 4, 9, 25, 41, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77]:
    create_jsonl(create_json_object(i), 'ft_0913.jsonl')

Upload the prepared dataset

In [27]:

file = openai.File.create(
    file=open("ft_0913.jsonl", "rb"),
    purpose='fine-tune'
)
print(file.id)

file-cOGsxBJQUC92HHC8qQ1I5cKd


Start Fine-tune job

In [28]:

openai.FineTune.list()
ft_obj = openai.FineTuningJob.create(training_file=file.id, model="gpt-3.5-turbo-0613")
print(ft_obj)

{
  "object": "fine_tuning.job",
  "id": "ftjob-NWNQIxLKjF5PvXYGQaMH6wsw",
  "model": "gpt-3.5-turbo-0613",
  "created_at": 1694588174,
  "finished_at": null,
  "fine_tuned_model": null,
  "organization_id": "org-0n31SOlYOG0jXlq6NaGViQdx",
  "result_files": [],
  "status": "created",
  "validation_file": null,
  "training_file": "file-cOGsxBJQUC92HHC8qQ1I5cKd",
  "hyperparameters": {
    "n_epochs": 4
  },
  "trained_tokens": null,
  "error": null
}


Wait for fine-tune job to be finished. When finished, the model name will be listed: e.g. davinci:ft-personal-2023-08-15-02-12-49


In [38]:
print(openai.FineTuningJob.retrieve(id=ft_obj.id).status)
ft_model = openai.FineTuningJob.retrieve(id=ft_obj.id).fine_tuned_model
print(ft_model)

succeeded
ft:gpt-3.5-turbo-0613:phase-software::7yEFQItN


Use the newly trained model

In [39]:
def apply_ft(description, model_name):
    completion = openai.ChatCompletion.create(
        model=model_name,
        messages=[
            {"role": "system", "content": "You are lottie file generator. lottie file should be inside code block ```json ```. format of lottie file: version is 5.12.1. layer is required. one shape should be in layer. a, p, o, r, s should be in both layer and shape,  hd = false. ip, op, sr, st should be in top level, layer, and shape. fill and transform should be parallel to shape json. for keyframe,  it should has two json objects with field i,o,s,t,ti,to."},
            {"role": "user", "content": description}
        ]
    )
    return completion.choices[0].message.content


Examples of the result: 

In [54]:
out = apply_ft("generate lottie file pink circle move right to left in 3sec", ft_model)
with open('ft_pink_circle.json', 'w') as outfile:
    outfile.write(out)

In [59]:
out = apply_ft("generate lottie file red rectangle grow", ft_model)
with open('ft_red_rect_grow.json', 'w') as outfile:
    outfile.write(out)

In [64]:
out = apply_ft("generate lottie file blue rect move [0.0] to [50, 50] to [100, 50] to [0.0]", ft_model)
with open('ft_blue_rect_move_corners.json', 'w') as outfile:
    outfile.write(out)

In [65]:
out = apply_ft("generate lottie file red circle move around 4 corners", ft_model)
with open('ft_red_circle_move_corners.json', 'w') as outfile:
    outfile.write(out)

In [62]:
out = apply_ft("generate lottie file orangle circle flip", ft_model)
with open('ft_orangle_circle_flip.json', 'w') as outfile:
    outfile.write(out)

#### Conclusion 

Even with a limited set of approximately 25 samples, fine-tuning has resulted in noticeable enhancements—eliminating the need for lengthy instructions to teach the Language Learning Model (LLM) the correct format. However, the current approach only allows for basic level instructions, such as converting "flip square" into a keyframe description like "width 100% to 0% to 100%." One avenue for future exploration could be the development of a separate model tasked with translating human language into keyframe-based descriptions. These descriptions could then be used to generate JSON files, potentially boosting performance. Another interesting direction would be to experiment with multiple shapes and varying instructions to see if this leads to further improvements.